In [1]:
# Author: Claire Wagner
# Date: 15 June 2022
# Purpose: To generate data about city-owned PINs that are of interest to HEI because they are in Ward 20 and/or have the zip code 60637 

In [2]:
import pandas as pd
import urllib.parse
import datetime

In [3]:
def makeAPIRequest(api_endpoint, params, limit, read_function):
    """Helper function to make Socrata API request."""
    query = "?" + "&".join(params) + "&$limit=" + str(limit)
    return read_function(api_endpoint + urllib.parse.quote(query, safe="&?$=,!()"))

In [4]:
# fetch data from the City-Owned Land Inventory dataset about all properties currently owned and/or leased by the City of Chicago (excluding obsolete PINs)
city_owned = makeAPIRequest(
    api_endpoint = "https://data.cityofchicago.org/resource/aksk-kvfp.json",
    params = [
        "$select=pin, managing_organization, lower(property_status) AS property_status, date_of_acquisition, date_of_disposition, sq_ft, last_update, :created_at, :updated_at",
        "$where=lower(property_status) in ('owned by city', 'leased') AND lower(managing_organization)!='obsolete pin'",
    ],
    limit = 100000,
    read_function = pd.read_json,
)

In [5]:
# fetch data from the Property Locations dataset about all properties that have zip code 60637 and/or are located in Ward 20
properties = makeAPIRequest(
    api_endpoint = "https://datacatalog.cookcountyil.gov/resource/c49d-89sn.json",
    params = [
        "$select=pin, property_address, property_zip, ward, longitude, latitude, :created_at, :updated_at",
        "$where=starts_with(property_zip, '60637') OR ward='20'",
    ],
    limit = 100000,
    read_function = pd.read_json,
)

In [11]:
# Generate attribution string that gives the URLs and the access date and time for the data sources.
fetchtime = datetime.datetime.now(datetime.timezone.utc).strftime("%d %B %Y at roughly %H:%M UTC")
attribution = f"/* Data sources: https://data.cityofchicago.org/Community-Economic-Development/City-Owned-Land-Inventory/aksk-kvfp and https://datacatalog.cookcountyil.gov/Property-Taxation/Assessor-Archived-05-11-2022-Property-Locations/c49d-89sn, accessed on {fetchtime}. */"

In [12]:
# prepare to compute the inner join of these two datasets by standardizing the PIN format
city_owned['pin'] = city_owned['pin'].str.replace('-','') 
properties['pin'] = properties['pin'].apply(str)

In [13]:
# compute the inner join (i.e. find all city-owned properties that have the zip code 60637 and/or are in Ward 20)
pins_to_map = pd.merge(city_owned, properties, how="inner", on="pin", suffixes = ["_aksk-kvfp", "_c49d-89sn"])
pins_to_map = pins_to_map.set_index('pin')

In [14]:
# output data to "data.js"
with open("data.js", "w", encoding="utf-8") as f:
    f.write(attribution + "\n") # include attribution
    f.write("const data = ") # assign JSON data to variable for easier access
    f.write(pins_to_map.to_json(orient="index")) # output pins_to_map as JSON